# Reading in Urban Observatory rain gauge data 

### Notes
- 


### What does the code do

### What does the code need to do?
1. Download the data
    - Try historic API
    - Try real-time API
    - Save data
2. Quality control data
    - Use intense-qc code
    - Remove failed gauges
    - Remove failed observations
    - Save Qc'ed data
3. Grid data
    - Save gridded data
4. Clip to domains
    - SHETRAN 
    - HIPIMS
    - Save

In [1]:
# Import relevent packages
import pandas as pd
import numpy as np
import requests
from os.path import join, exists
import os
from datetime import datetime
import io

In [2]:
##### Inputs to change
start_date = "2022-06-01"
end_date = "2022-06-16"

out_path = r"C:\Users\Amy\OneDrive - Newcastle University\Documents\PYRAMID\data\event\test"

# Bounding box for data 
e_l, n_l, e_u, n_u = [355000, 534000, 440000, 609000]
bbox = [e_l, e_u, n_l, n_u]

In [3]:
uo_outpath = join(out_path, "UO")
if not exists(uo_outpath):
    os.mkdir(uo_outpath)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Amy\\OneDrive - Newcastle University\\Documents\\PYRAMID\\data\\event\\test\\UO'

In [ ]:
# Get list of rainfall stations
root = "http://environment.data.gov.uk/flood-monitoring"
response = requests.get(root + "/id/stations?parameter=rainfall")
r = response.json()["items"]
df = pd.json_normalize(r)

In [15]:
ea_outpath = join(out_path, "EA")
if not exists(ea_outpath):
    os.mkdir(ea_outpath)

# Get list of rainfall stations
root = "http://environment.data.gov.uk/flood-monitoring"
response = requests.get(root + "/id/stations?parameter=rainfall")
r = response.json()["items"]
df = pd.json_normalize(r)

# Get stations in area
extent_stations = df.loc[(df.northing > n_l) & (df.northing < n_u) & (df.easting > e_l) & (df.easting < e_u)]
station_ids = extent_stations.stationReference.to_list()
names = np.array(extent_stations.stationReference + "_" + extent_stations.easting.astype(int).astype(
    str) + "_" + extent_stations.northing.astype(int).astype(str) + ".csv")

now = datetime.now()
# use real-time API if possible (last 28 days)
if now - pd.to_datetime(start_date) < pd.Timedelta("28d"):
    for i, station in enumerate(station_ids):
        try:
            path = "http://environment.data.gov.uk/flood-monitoring/id/measures/{id}-rainfall-tipping_bucket_raingauge-t-15_min-mm/readings?parameter=rainfall&startdate={startDate}&enddate={endDate}"
            path = path.replace("{id}", station)
            path = path.replace("{startDate}", start_date)
            path = path.replace("{endDate}", end_date)
            path = path.replace("{startTime}", start_date)

            response = requests.get(path)
            r = response.json()["items"]
            df = pd.json_normalize(r)
            if len(df) > 0:
                data = pd.Series(df.value.values, index=pd.to_datetime(df.dateTime))
                data.to_csv(join(ea_outpath, names[i]))
        except:
            print(station, "not worked.")

# if not use historical API (seems to only work for last year- not very historical)
else:
    dates = pd.date_range(start_date, end_date)
    iids = np.array(extent_stations.measures.str[0].str["@id"])

    for date_ts in dates:
        date = str(date_ts).split(" ")[0]
        path = "http://environment.data.gov.uk/flood-monitoring/archive/readings-full-{date}.csv"
        path = path.replace("{date}", str(date))
        r = requests.get(path)
        full_data = []
        if r.status_code == 200:
            df = pd.read_csv(io.StringIO(r.text), index_col=0, parse_dates=True)
            full_data.append(df[[measure in iids for measure in df.measure]])
    full_data = pd.concat(full_data).groupby("stationReference")

    for i, station in enumerate(station_ids):
        try:
            full_data.get_group(station).value.to_csv(join(ea_outpath, names[i]))
        except:
            print(station, "not worked.")

C:\Users\Amy\AppData\Local\Temp\ipykernel_14312\835027576.py:49: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(r.text), index_col=0, parse_dates=True)
C:\Users\Amy\AppData\Local\Temp\ipykernel_14312\835027576.py:49: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(r.text), index_col=0, parse_dates=True)


worked
worked
worked
worked
worked
worked
worked
worked
worked
019356 not worked.
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
007533 not worked.
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked


In [46]:
# Import relevent packages
import os
from os.path import join
import pandas as pd
#os.sys.path.append(r"C:\Users\Amy\OneDrive - Newcastle University\Documents\Jupyter\intense-qc")
from intense import gauge, qc, utils
from pyproj import Transformer

In [40]:
### Inputs to change
out_path = r"C:\Users\Amy\OneDrive - Newcastle University\Documents\PYRAMID\data\event\test"
gauge_path = join(out_path, "EA")
etccdi_data_folder = r"C:\Users\Amy\OneDrive - Newcastle University\Documents\Jupyter\intense-qc\tests\etccdi_data"

gauge_files = [f for f in os.listdir(gauge_path) if f.endswith(".csv")]

gauges = {}

for g in gauge_files:
    name = g[0:-4]
    data = pd.read_csv(join(gauge_path, g), index_col=0, parse_dates=True).iloc[:, 0]
    data = data.sort_index()
    res_min = pd.Series(data.dropna().index).diff().median().seconds / 60
    
    try:
        if data.index.dtype == "datetime64[ns, UTC]":
            gauges[name] = data.resample("1h").sum()
        else:
            gauges[name] = data.resample("1h").sum().tz_localize('UTC')
    except:
        print(name, "not read in.")

gauges = pd.DataFrame(gauges)
gauges = gauges.dropna(axis=0, how="all")

In [44]:
def get_gauge_flags(data, loc):
    '''Function to apply wokring Intense QC hourly checks on gauge data. 
    Args: data is the gauge data (pd.Series), loc is the gauge location (eastings, northings)
    Returns: flags as a dictionary {gauge flagged, years flagged, obs flagged}
    '''
    eastings, northings = loc
    # convert coordinates to lat/lon
    transformer = Transformer.from_crs("epsg:27700", "epsg:4326")
    latitude, longitude = transformer.transform(eastings, northings)

    # create gauge object
    rain_gauge = gauge.Gauge(
        station_id,
        path_to_original_data="",
        latitude=latitude,
        longitude=longitude,
        original_timestep="15min",
        original_units="mm/h",
        new_units="mm/h",
        new_timestep="1h",
        data=data
    )
    rain_gauge.get_info()

    # create qc object 
    test = qc.Qc(
        gauge=rain_gauge,
        etccdi_data_folder=etccdi_data_folder
    )

    # checks that don't work on existing data
    """
    test.check_percentiles()
    test.check_k_largest()
    test.check_intermittency()
    test.change_in_min_val_check() 
    test.cwd_check() # missing function in utils file???
    test.change_in_min_val_check() # Change in minimum value check, homogeneity check to see if the resolution of the data has changed. Change flag, flag years
    test.find_neighbours("hourly") # frequency: must be either hourly, daily or monthly, Names or names and paths of neighbouring stations
    # conditions are: must be within 50km, at least 3 years overlap, select the closest 10, don't have three years of data
    # check_hourly_neighbours(), check_daily_neighbours(), check_monthly_neighbours()
    test.get_flags() # runs all checks, fails at find_neighbours()
    """

    ### run checks ###

    flagged_sdii = 0
    sdii_thresh = 100 # just arbitrary atm
    if any(np.array(test.get_sdii()) > sdii_thresh): # Simple precipitation intensity index, SDII from ETCCDI and from gauge values (sdii_gridded, sdii_gauge), not sure how to use this
        flagged_sdii = 1
    
    # Flag data if any of these don't return 0:
    gauge_checks = [
        test.check_days_of_week(), # Checks if proportions of rainfall in each day is significantly different
        test.check_break_point(), # Pettitt breakpoint check
        flagged_sdii
    ]
    
    flagged_gauge = sum(gauge_checks) != 0
    #if flagged_gauge:
    #    print("Gauge", np.array(["days of week", "break point", "sdii"])[np.array(gauge_checks) == 1])
    #    print(test.check_break_point())

    # Flag individual data observations if don't return 0:
    obs_checks = pd.DataFrame(index=rain_gauge.data.index)
    obs_checks["world_record"] = test.world_record_check_ts() # Checks if and to what degree the world record has been exceeded by each rainfall value, 4, 3, 2 or 1 if exceeded by > 1.5x, 1.33x, 1.22x or 0x respectively and 0 if not exceeded for each value
    obs_checks["rx1day"] = test.rx1day_check_ts() # Checks hourly values against maximum 1-day precipitation, Magnitudes of exceedance for each day
    obs_checks["cdd"] = test.cdd_check() # ETCCDI provide an index for maximum length of dry spell. Look for suspicious number of consecutive dry hours recorded. Consecutive Dry Days: Maximum length of dry spell, maximum number of consecutive days with RR < 1mm. Magnitudes of exceedence of the length of longest dry period
    obs_checks["daily_accums"] = test.daily_accums_check() # Check daily accumulations. Suspect daily accumulations flagged where a recorded rainfall amount at these times is preceded by 23 hours with no rain. A threshold of 2x the mean wet day amount for the corresponding month is applied to increase thechance of identifying accumulated values at the expense of genuine, moderate events
    obs_checks["monthly_accums"] = test.monthly_accums_check() # Check monthly accumulations. Flags month prior to high value
    obs_checks["streaks"] = test.streaks_check() # Streaks: This is where you see the same value repeated in a run. Currently this records streaks of 2hrs in a row or more over 2 x Monthly mean rainfall. It is considered to be unlikely that you would see even 2 consecutive large rainfall amounts. For this code I have substituted the monthly mean rainfall for SDII as I want the thresholds to be independent of the rainfall time series as the global dataset is of highly variable quality.
    flagged_obs = obs_checks.index[obs_checks.sum(1) > 0]

    # Flags each individual year:
    year_flags = np.array([
        test.r99ptot_check_annual(), # Check against R99pTOT: R99pTOT. Annual total PRCP when RR > 99p. Magnitudes of exceedance for yearly 99th percentiles
        test.prcptot_check_annual() # check against annual total: PRCPTOT. Annual total precipitation in wet days. Magnitudes of exceedance for yearly totals
    ])

    flagged_years = np.arange(rain_gauge.data.index.min().year, rain_gauge.data.index.max().year + 1)[year_flags.sum(0) < 0]

    return({"gauge" : flagged_gauge, "years" : flagged_years, "obs" : flagged_obs})

In [47]:
qc_gauges = pd.DataFrame(index=gauges.index)
for i in range(len(gauges.columns)):
    col = gauges.columns[i]
    station_id, eastings, northings = col.split("_")

    data = gauges.iloc[:, i].dropna()
    loc = (eastings, northings)

    flags = get_gauge_flags(data, loc)

    # only include gauge data that is not flagged
    if not flags["gauge"]:

        # currenly ignoring year flag as not enough data

        # remove flagged observationns
        if len(flags["obs"]) < 0:
            qc_gauges[col] = gauges[col]
        else:
            obs_cond = [t not in flags["obs"] for t in gauges.index]
            qc_gauges[col] = np.nan
            
            qc_gauges.loc[obs_cond, col] = gauges.loc[obs_cond, col]

# gauges failing on break point

C:\Users\Amy\OneDrive - Newcastle University\Documents\Jupyter\intense-qc\intense\qc.py:600: RuntimeWarning: invalid value encountered in double_scalars
  sdii_gauge = prcp_sum / float(wetday_count)
C:\Users\Amy\anaconda3\envs\wradlib\lib\site-packages\scipy\stats\_stats_py.py:1250: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
C:\Users\Amy\anaconda3\envs\wradlib\lib\site-packages\scipy\stats\_stats_py.py:1250: RuntimeWarning: invalid value encountered in double_scalars
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
C:\Users\Amy\OneDrive - Newcastle University\Documents\Jupyter\intense-qc\intense\qc.py:600: RuntimeWarning: invalid value encountered in double_scalars
  sdii_gauge = prcp_sum / float(wetday_count)
C:\Users\Amy\OneDrive - Newcastle University\Documents\Jupyter\intense-qc\intense\qc.py:600: RuntimeWarning: invalid value encountered in double_scalars
  sdii_gauge = prcp_sum / floa

C:\Users\Amy\OneDrive - Newcastle University\Documents\Jupyter\intense-qc\intense\qc.py:600: RuntimeWarning: invalid value encountered in double_scalars
  sdii_gauge = prcp_sum / float(wetday_count)
C:\Users\Amy\anaconda3\envs\wradlib\lib\site-packages\scipy\stats\_stats_py.py:1250: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
C:\Users\Amy\anaconda3\envs\wradlib\lib\site-packages\scipy\stats\_stats_py.py:1250: RuntimeWarning: invalid value encountered in double_scalars
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
C:\Users\Amy\OneDrive - Newcastle University\Documents\Jupyter\intense-qc\intense\qc.py:600: RuntimeWarning: invalid value encountered in double_scalars
  sdii_gauge = prcp_sum / float(wetday_count)
C:\Users\Amy\OneDrive - Newcastle University\Documents\Jupyter\intense-qc\intense\qc.py:600: RuntimeWarning: invalid value encountered in double_scalars
  sdii_gauge = prcp_sum / floa

C:\Users\Amy\OneDrive - Newcastle University\Documents\Jupyter\intense-qc\intense\qc.py:600: RuntimeWarning: invalid value encountered in double_scalars
  sdii_gauge = prcp_sum / float(wetday_count)
C:\Users\Amy\anaconda3\envs\wradlib\lib\site-packages\scipy\stats\_stats_py.py:1250: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
C:\Users\Amy\anaconda3\envs\wradlib\lib\site-packages\scipy\stats\_stats_py.py:1250: RuntimeWarning: invalid value encountered in double_scalars
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
C:\Users\Amy\OneDrive - Newcastle University\Documents\Jupyter\intense-qc\intense\qc.py:600: RuntimeWarning: invalid value encountered in double_scalars
  sdii_gauge = prcp_sum / float(wetday_count)
C:\Users\Amy\OneDrive - Newcastle University\Documents\Jupyter\intense-qc\intense\qc.py:600: RuntimeWarning: invalid value encountered in double_scalars
  sdii_gauge = prcp_sum / floa

C:\Users\Amy\OneDrive - Newcastle University\Documents\Jupyter\intense-qc\intense\qc.py:600: RuntimeWarning: invalid value encountered in double_scalars
  sdii_gauge = prcp_sum / float(wetday_count)
C:\Users\Amy\anaconda3\envs\wradlib\lib\site-packages\scipy\stats\_stats_py.py:1250: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
C:\Users\Amy\anaconda3\envs\wradlib\lib\site-packages\scipy\stats\_stats_py.py:1250: RuntimeWarning: invalid value encountered in double_scalars
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
C:\Users\Amy\OneDrive - Newcastle University\Documents\Jupyter\intense-qc\intense\qc.py:600: RuntimeWarning: invalid value encountered in double_scalars
  sdii_gauge = prcp_sum / float(wetday_count)
C:\Users\Amy\OneDrive - Newcastle University\Documents\Jupyter\intense-qc\intense\qc.py:600: RuntimeWarning: invalid value encountered in double_scalars
  sdii_gauge = prcp_sum / floa

C:\Users\Amy\OneDrive - Newcastle University\Documents\Jupyter\intense-qc\intense\qc.py:600: RuntimeWarning: invalid value encountered in double_scalars
  sdii_gauge = prcp_sum / float(wetday_count)
C:\Users\Amy\anaconda3\envs\wradlib\lib\site-packages\scipy\stats\_stats_py.py:1250: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
C:\Users\Amy\anaconda3\envs\wradlib\lib\site-packages\scipy\stats\_stats_py.py:1250: RuntimeWarning: invalid value encountered in double_scalars
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
C:\Users\Amy\OneDrive - Newcastle University\Documents\Jupyter\intense-qc\intense\qc.py:600: RuntimeWarning: invalid value encountered in double_scalars
  sdii_gauge = prcp_sum / float(wetday_count)
C:\Users\Amy\OneDrive - Newcastle University\Documents\Jupyter\intense-qc\intense\qc.py:600: RuntimeWarning: invalid value encountered in double_scalars
  sdii_gauge = prcp_sum / floa

C:\Users\Amy\OneDrive - Newcastle University\Documents\Jupyter\intense-qc\intense\qc.py:600: RuntimeWarning: invalid value encountered in double_scalars
  sdii_gauge = prcp_sum / float(wetday_count)
C:\Users\Amy\anaconda3\envs\wradlib\lib\site-packages\scipy\stats\_stats_py.py:1250: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
C:\Users\Amy\anaconda3\envs\wradlib\lib\site-packages\scipy\stats\_stats_py.py:1250: RuntimeWarning: invalid value encountered in double_scalars
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
C:\Users\Amy\OneDrive - Newcastle University\Documents\Jupyter\intense-qc\intense\qc.py:600: RuntimeWarning: invalid value encountered in double_scalars
  sdii_gauge = prcp_sum / float(wetday_count)
C:\Users\Amy\OneDrive - Newcastle University\Documents\Jupyter\intense-qc\intense\qc.py:600: RuntimeWarning: invalid value encountered in double_scalars
  sdii_gauge = prcp_sum / floa

In [ ]:
ea_inpath = join(out_path, "EA")
ea_outpath = join(out_path, "EA")
start_date = "2022-06-01"
end_date = "2022-06-16"

start_time = pd.to_datetime(start_date).round(str(15*60) + "s")
end_time = pd.to_datetime(end_date).round(str(15*60) + "s")


gauge_colnames = [f.split(".")[0] for f in os.listdir(ea_inpath) if f.endswith(".csv")]

ea_gauges = pd.DataFrame(index=pd.date_range(start_time, end_time, freq=str(15*60) + "s"))
g_x = np.empty(len(gauge_colnames))
g_y = np.empty(len(gauge_colnames))

for i, f in enumerate(gauge_colnames):

In [84]:
ea_inpath = join(out_path, "EA")
ea_outpath = join(out_path, "EA")
start_date = "2022-06-01"
end_date = "2022-06-16"

start_time = pd.to_datetime(start_date).round(str(15*60) + "s")
end_time = pd.to_datetime(end_date).round(str(15*60) + "s")


gauge_colnames = [f.split(".")[0] for f in os.listdir(ea_inpath) if f.endswith(".csv")]

ea_gauges = pd.DataFrame(index=pd.date_range(start_time, end_time, freq=str(15*60) + "s"))
g_x = np.empty(len(gauge_colnames))
g_y = np.empty(len(gauge_colnames))

for i, f in enumerate(gauge_colnames):

    g_x[i] = (f.split("_")[1])
    g_y[i] = float(f.split("_")[2])
    gauge = pd.read_csv(join(ea_inpath, f + ".csv"), index_col=0)
    ea_gauges[f] = np.nan
    ea_gauges.loc[pd.to_datetime(gauge.index), f] = gauge.values 
    

KeyError: "None of [DatetimeIndex(['2023-03-01 05:00:00+00:00', '2023-03-01 04:15:00+00:00',\n               '2023-03-01 02:45:00+00:00', '2023-03-01 03:30:00+00:00',\n               '2023-03-01 00:00:00+00:00', '2023-03-01 04:00:00+00:00',\n               '2023-03-01 04:30:00+00:00', '2023-03-01 03:15:00+00:00',\n               '2023-03-01 01:30:00+00:00', '2023-03-01 02:15:00+00:00',\n               ...\n               '2023-03-04 18:30:00+00:00', '2023-03-04 20:15:00+00:00',\n               '2023-03-04 21:00:00+00:00', '2023-03-04 18:00:00+00:00',\n               '2023-03-04 19:15:00+00:00', '2023-03-04 14:15:00+00:00',\n               '2023-03-04 07:30:00+00:00', '2023-03-04 13:00:00+00:00',\n               '2023-03-04 14:45:00+00:00', '2023-03-04 07:00:00+00:00'],\n              dtype='datetime64[ns, UTC]', name='dateTime', length=384, freq=None)] are in the [index]"

In [83]:
ea_gauges

,003066_389250_606250
2022-06-01 00:00:00,NaN
2022-06-01 00:15:00,NaN
2022-06-01 00:30:00,NaN
2022-06-01 00:45:00,NaN
2022-06-01 01:00:00,NaN
...,...
2022-06-15 23:00:00,NaN
2022-06-15 23:15:00,NaN
2022-06-15 23:30:00,NaN
2022-06-15 23:45:00,NaN
